<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/preprocess_recipeQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Mounted at /content/gdrive


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

100% 2.54G/2.55G [00:37<00:00, 27.3MB/s]
100% 2.55G/2.55G [00:37<00:00, 72.4MB/s]


In [13]:
import json
import numpy as np
import tensorflow as tf

def read_file(file_name):
  with open(file_name) as f:
    data = json.load(f)['data']
  textual_cloze = [item for item in data if item["task"]=="textual_cloze"]
  visual_cloze = [item for item in data if item["task"]=="visual_cloze"]
  visual_coherence = [item for item in data if item["task"]=="visual_coherence"]
  visual_ordering = [item for item in data if item["task"]=="visual_ordering"]
  return textual_cloze, visual_cloze, visual_coherence, visual_ordering

In [14]:
def delete_keys(dataset):
  if (dataset[0]["task"]=="visual_coherence"):
    [data.pop("question") for data in dataset]
  if (dataset[0]["task"]=="visual_ordering"):
    [data.pop("question") for data in dataset]
    for data in dataset:
      order = data["choice_list"][0]
      data["image_list"] = {0:order[0], 1:order[1], 2:order[2], 3:order[3]}
      img2ind = {v:k for k,v in data["image_list"].items()}
      data["choice_list"]=[[img2ind[img] for img in choice] for choice in data["choice_list"]]
  [[data.pop(key,None) for key in ["context_modality", "split", "qid", "question_modality", "task", "question_text"]] for data in dataset]
  [[[step.pop(key) for key in ["id","videos"]] for step in data['context']] for data in dataset]
#  print(json.dumps(dataset[0], indent=4))

In [15]:
def load_image(file_path):
# Load image
  image = tf.io.decode_jpeg(tf.io.read_file(file_path),channels=3)
# Convert image to normalized float [0, 1]
  image = tf.image.convert_image_dtype(image,tf.float32)
# resize image
  image = tf.image.resize(image, [256,256])
# Rescale data to range (-1, 1)
  image = (image - 0.5) * 2
  return image

In [16]:
from copy import deepcopy
def data_iter(batch_size, dataset, task, split):
  num_input = len(dataset)
  np.random.shuffle(dataset)
  for i in range(num_input // batch_size):
    Xs = deepcopy(dataset[i*batch_size:(i+1)*batch_size])
    Ys = [item.pop("answer") for item in Xs]
    if task=="textual_cloze":
      for X in Xs:
        for step in X["context"]:
          step["images"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in step["images"]]
    if task=="visual_cloze":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
        X["question"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) if not item=="@placeholder" else "@placeholder"  for item in X["question"]]
    if task=="visual_coherence":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
    if task=="visual_ordering":
      for X in Xs:
        for k,v in X["image_list"].items():
          X["image_list"][k] = load_image("data/images/images-qa/"+split+"/images-qa/"+X["image_list"][k])
    yield Xs, Ys



In [17]:
def preprocess(batch_size, split):
  textual_cloze, visual_cloze, visual_coherence, visual_ordering=read_file("data/"+split+" recipeqa.json")
  delete_keys(textual_cloze)
  delete_keys(visual_cloze)
  delete_keys(visual_coherence)
  delete_keys(visual_ordering)
  textual_cloze_iter = data_iter(batch_size, textual_cloze, "textual_cloze", split)
  visual_cloze_iter = data_iter(batch_size, visual_cloze, "visual_cloze", split)
  visual_coherence_iter = data_iter(batch_size, visual_coherence, "visual_coherence", split)
  visual_ordering_iter = data_iter(batch_size, visual_ordering, "visual_ordering", split)
  return textual_cloze_iter, visual_cloze_iter, visual_coherence_iter, visual_ordering_iter

In [18]:
batch_size = 50
train_it1, train_it2, train_it3, train_it4 = preprocess(batch_size, "train")
test_it1, test_it2, test_it3, test_it4 = preprocess(batch_size, "test")
val_it1, val_it2, val_it3, val_it4 = preprocess(batch_size, "val")

In [33]:
# Usage:
for Xs, Ys in val_it2:
  print(json.dumps(Xs[0], indent=4, default=lambda x:"tf_tensor"))
  print(Ys[0])
  break

{
    "recipe_id": "old-fashioned-macaroni-and-cheese",
    "context": [
        {
            "body": "- 6 to 7 ounces raw elbow macaroni (about 2 cups)- 3 cups shredded cheddar cheese (about 3/4 pounds)- 1 jar of alfredo (Ragu~classic alfredo)- Milk- Salt- Pepper- 2-quart casserole dish- Measuring cups- Stove top pans",
            "title": "Supplies Needed"
        },
        {
            "body": "Preheat the oven to 375 degrees.",
            "title": "Preheat Oven"
        },
        {
            "body": "Boil the elbow noodles on a stove top as instructed on their package. Once they are done cooking place half of the amount of noodles into an ungreased 2-quart casserole dish.",
            "title": "Cook Noodles"
        },
        {
            "body": "On the side start making your white sauce, put the one full jar of alfredo(Ragu) into the oven to pan, fill the empty alfredo jar up with milk and pour that into the pan also. Bring to a boil over medium temperature... stir fre